In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import numpy as np
import sklearn

from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.datasets import make_regression
from sklearn.decomposition import SparseCoder, sparse_encode
from sklearn.linear_model import orthogonal_mp_gram

import scipy.linalg as LA

# Sklearn

## sparse_encode

In [2]:
def sklearn_OMP(Y, T_0, D, rng=42):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    # X = SparseCoder(dictionary = D, transform_n_nonzero_coefs = T_0).transform(Y)
    gram = D @ D.T
    cov = D @ Y.T
    X = sparse_encode(Y, 
                      D, 
                      algorithm='omp', 
                      n_nonzero_coefs = T_0,
                      gram = gram,
                      cov = cov,
                      copy_cov=False)
    return X

## orthogonal_mp_gram

In [3]:
def sklearn_OMP2(Y, T_0, D, rng=42):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    # X = SparseCoder(dictionary = D, transform_n_nonzero_coefs = T_0).transform(Y)
    gram = D @ D.T
    cov = D @ Y.T
    X = orthogonal_mp_gram(
        Gram=gram, 
        Xy=cov, 
        n_nonzero_coefs=T_0, 
        copy_Gram=False, 
        copy_Xy=False
    ).T
    return X

# Numpy lstsq

In [4]:
def OMP(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    # gram = D @ D.T
    # cov = D @ Y.T
    # X = sparse_encode(Y, 
    #                   D, 
    #                   algorithm='omp', 
    #                   n_nonzero_coefs = T_0,
    #                   gram = gram,
    #                   cov = cov,
    #                   copy_cov=False)

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0

        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            gamma, res, rank, s = np.linalg.lstsq(D_I[:j+1].T, y)
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - D_I[:(j+1)].T @ gamma 
            if debug:
                print(np.sum(r * r))
                print(res)

        X[i, I] = gamma
        
    return X

# Scipy lstsq (gelsd driver)

In [5]:
def OMP_2(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    # gram = D @ D.T
    # cov = D @ Y.T
    # X = sparse_encode(Y, 
    #                   D, 
    #                   algorithm='omp', 
    #                   n_nonzero_coefs = T_0,
    #                   gram = gram,
    #                   cov = cov,
    #                   copy_cov=False)

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0

        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            gamma, res, rank, s = LA.lstsq(D_I[:j+1].T, y, lapack_driver='gelsd', check_finite=False)
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - D_I[:(j+1)].T @ gamma 
            if debug:
                print(np.sum(r * r))
                print(res)

        X[i, I] = gamma
        
    return X

# Scipy lstsq (gelsy driver)

In [6]:
def OMP_3(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    # gram = D @ D.T
    # cov = D @ Y.T
    # X = sparse_encode(Y, 
    #                   D, 
    #                   algorithm='omp', 
    #                   n_nonzero_coefs = T_0,
    #                   gram = gram,
    #                   cov = cov,
    #                   copy_cov=False)

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0

        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            gamma, res, rank, s = LA.lstsq(D_I[:j+1].T, y, lapack_driver='gelsy', check_finite=False)
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - D_I[:(j+1)].T @ gamma 
            if debug:
                print(np.sum(r * r))
                print(res)

        X[i, I] = gamma
        
    return X

In [7]:
def OMP_verif(code):
    print(code)
    I = np.argmax(np.abs(code), axis = 1)
    print(I)
    print(code[np.arange(I.shape[0]), I])

In [8]:
X, y = make_regression(n_samples = 50, n_features = 20, n_targets = 2, noise=4, random_state=0)

In [9]:
X.shape

(50, 20)

In [10]:
y.shape

(50, 2)

In [11]:
b_sklearn = sklearn_OMP(y.T, 1, X.T)

In [12]:
OMP_verif(b_sklearn)

[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.          99.70035543   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         146.54061532   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
[11  1]
[ 99.70035543 146.54061532]


In [13]:
b_sklearn2 = sklearn_OMP2(y.T, 1, X.T)

In [14]:
OMP_verif(b_sklearn2)

[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.          99.70035543   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         146.54061532   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
[11  1]
[ 99.70035543 146.54061532]


In [15]:
b_numpy = OMP(y.T, 1, X.T, debug=True)

(1,)
(50, 1)
(50,)
2449425.0712439856
[2449425.07124399]
(1,)
(50, 1)
(50,)
1817892.1699581572
[1817892.16995816]


In [16]:
OMP_verif(b_numpy)

[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.          99.70035543   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         146.54061532   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
[11  1]
[ 99.70035543 146.54061532]


In [17]:
b_scipy = OMP_2(y.T, 1, X.T, debug=True)

(1,)
(50, 1)
(50,)
2449425.0712439856
2449425.071243986
(1,)
(50, 1)
(50,)
1817892.1699581572
1817892.1699581575


In [18]:
OMP_verif(b_scipy)

[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.          99.70035543   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         146.54061532   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
[11  1]
[ 99.70035543 146.54061532]


In [19]:
b_scipy2 = OMP_3(y.T, 1, X.T, debug=True)

(1,)
(50, 1)
(50,)
2449425.071243986
[]
(1,)
(50, 1)
(50,)
1817892.1699581572
[]


In [20]:
OMP_verif(b_scipy2)

[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.          99.70035543   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         146.54061532   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
[11  1]
[ 99.70035543 146.54061532]


In [21]:
np.allclose(b_sklearn2[[0, 1], [11, 1]], b_sklearn[[0, 1], [11, 1]])

True

In [22]:
np.allclose(b_numpy[[0, 1], [11, 1]], b_sklearn[[0, 1], [11, 1]])

True

In [23]:
np.allclose(b_scipy[[0, 1], [11, 1]], b_sklearn[[0, 1], [11, 1]])

True

In [24]:
np.allclose(b_scipy2[[0, 1], [11, 1]], b_sklearn[[0, 1], [11, 1]])

True

## IT WORKS !!!

In [25]:
X, y = make_regression(n_samples = 50, n_features = 300, n_targets = 10_000, noise=4, random_state=0)

In [26]:
%timeit sklearn_OMP(y.T, 1, X.T)

632 ms ± 93.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit sklearn_OMP2(y.T, 1, X.T)

682 ms ± 52.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit OMP(y.T, 1, X.T)

547 ms ± 9.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%timeit OMP_2(y.T, 1, X.T)

811 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%timeit OMP_3(y.T, 1, X.T)

621 ms ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%load_ext line_profiler

In [32]:
from line_profiler import profile

sk_OMP = profile(sklearn_OMP)
%lprun -f sk_OMP sk_OMP(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.02785 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_25816\2677671288.py
Function: sklearn_OMP at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def sklearn_OMP(Y, T_0, D, rng=42):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5                                               # X = SparseCoder(dictionary = D, transform_n_nonzero_coefs = T_0).transform(Y)
     6         1       2906.0   2906.0      0.0      gram = D @ D.T
     7         1     160348.0 160348.0      1.6      cov = D @ Y.T
     8         2   10115192.0 5.06e+06     98.4      X = sparse_encode(Y, 
     9         1          2.0      2.0      0.0                        D, 
    10         1          3.0      3.0      0.0                      

In [33]:
from line_profiler import profile

sk_OMP2 = profile(sklearn_OMP2)
%lprun -f sk_OMP2 sk_OMP2(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.03095 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_25816\4239697825.py
Function: sklearn_OMP2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def sklearn_OMP2(Y, T_0, D, rng=42):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5                                               # X = SparseCoder(dictionary = D, transform_n_nonzero_coefs = T_0).transform(Y)
     6         1       3665.0   3665.0      0.0      gram = D @ D.T
     7         1     164309.0 164309.0      1.6      cov = D @ Y.T
     8         3   10141526.0 3.38e+06     98.4      X = orthogonal_mp_gram(
     9         1          3.0      3.0      0.0          Gram=gram, 
    10         1          3.0      3.0      0.0          Xy=cov, 
    1

In [34]:
np_OMP = profile(OMP)
%lprun -f np_OMP np_OMP(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 0.956548 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_25816\1184636726.py
Function: OMP at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1       2728.0   2728.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7                                               # gram = D @ D.T
     8                                               # cov = D @ Y.T
     9                                               # X = sparse_encode(Y, 
    10                                               #                   D, 
    11                                               #

In [35]:
sci_OMP = profile(OMP_2)
%lprun -f sci_OMP sci_OMP(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.45957 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_25816\1646719746.py
Function: OMP_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_2(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        343.0    343.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7                                               # gram = D @ D.T
     8                                               # cov = D @ Y.T
     9                                               # X = sparse_encode(Y, 
    10                                               #                   D, 
    11                                             

In [36]:
sci_OMP2 = profile(OMP_3)
%lprun -f sci_OMP2 sci_OMP2(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.24596 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_25816\1573553668.py
Function: OMP_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_3(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1       1066.0   1066.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7                                               # gram = D @ D.T
     8                                               # cov = D @ Y.T
     9                                               # X = sparse_encode(Y, 
    10                                               #                   D, 
    11                                             